<a href="https://colab.research.google.com/github/Chanda92/CE888/blob/master/Lab5/Recommender_System_Based_on_latent_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Chanda92/CE888/master/jester-data-1.csv',header=None)
print(df)

ERROR! Session/line number was not unique in database. History logging moved to new session 60
       0      1      2      3      4    ...    96     97     98     99     100
0       74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24978  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24979   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24980   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24982   72   2.43   2.67  -3.98   4.

In [3]:
df=df.drop(df.columns[0],1)
print(df)

         1      2      3      4     5    ...    96     97     98     99     100
0      -7.82   8.79  -9.66  -8.16 -7.52  ...  99.00  -5.63  99.00  99.00  99.00
1       4.08  -0.29   6.36   4.37 -2.38  ...  -2.14   3.06   0.34  -4.32   1.07
2      99.00  99.00  99.00  99.00  9.03  ...  99.00  99.00  99.00  99.00  99.00
3      99.00   8.35  99.00  99.00  1.80  ...  99.00  99.00  99.00  99.00  99.00
4       8.50   4.61  -4.17  -5.39  1.36  ...   1.55   3.11   6.55   1.80   1.60
...      ...    ...    ...    ...   ...  ...    ...    ...    ...    ...    ...
24978   0.44   7.43   9.08   2.33  3.20  ...   9.03   6.55   8.69   8.79   7.43
24979   9.13  -8.16   8.59   9.08  0.87  ...  -8.20  -7.23  -8.59   9.13   8.45
24980  99.00  99.00  99.00  99.00 -7.77  ...  99.00  99.00  99.00  99.00  99.00
24981  99.00  99.00  99.00  99.00 -9.71  ...  99.00  99.00  99.00  99.00  99.00
24982   2.43   2.67  -3.98   4.27 -2.28  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 100 columns]


In [4]:
arr = df.values
print(arr.shape)

(24983, 100)


In [5]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1810455 (1810455,)


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [0]:
arr[idx[0][0], idx[1][0]]

-9.37

In [8]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
n_latent_factors = 15

user_ratings = df.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=50):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = new_arr[user_id][item_id]
                if (rating!=99):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        #if (iteration % 10) == 0:
        print(mse)

In [11]:
sgd()

25.849500408670743
24.93266810310512
24.880988144096868
24.834415590759416
24.790546854639018
24.74746924453384
24.703345860816597
24.656096576635246
24.603041244245983
24.540454156040926
24.46298211205578
24.362900347872067
24.2292491480976
24.047064949811542
23.79726793840577
23.458313372776153
23.01118033398766
22.448692841480128
21.78702884309926
21.0718382818078
20.369511205269305
19.74363585653523
19.231874870770806
18.839561579877603
18.54995756312378
18.338755037245715
18.183498909937374
18.066877353220292
17.976600909503638
17.90422118139049
17.84396068927184
17.79183536466549
17.745056716168524
17.701637707681463
17.66013484551826
17.619479109319727
17.578864760757938
17.53767612240721
17.495439476482318
17.45179168143905
17.406459912261255
17.359248712094896
17.310031683880492
17.258745895878437
17.2053875834102
17.150008104232572
17.09270941188397
17.033638586606422
16.972981221186664
16.91095369645696


In [12]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[-2.649, -4.404, -3.868, ..., -6.116, -5.23 , -5.518],
       [ 3.668,  3.657,  3.64 , ...,  2.343,  2.071,  2.757],
       [ 6.855,  7.244,  6.918, ...,  7.775,  4.466,  8.515],
       ...,
       [-2.042, -0.802, -2.809, ..., -0.248, -4.084, -0.648],
       [-0.241, -0.737, -1.518, ...,  0.797, -1.917,  0.403],
       [ 3.556,  2.697,  3.045, ...,  3.544,  2.341,  4.18 ]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [14]:
comparison_data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, -2.649347974290913)","(8.79, -4.404108504323025)","(-9.66, -3.8681588540685925)","(-8.16, -7.834216169012298)","(-7.52, -2.973819271687742)","(-8.5, -3.421858359900176)","(-9.85, -3.3278045023938256)","(4.17, -1.5390689272008296)","(-8.98, -5.79684544907735)","(-4.76, -1.8940758663110364)","(-8.5, -3.2204245842412456)","(-6.75, -3.586797777104124)","(-7.18, -4.473096017746583)","(8.45, -4.084709337167269)","(-7.18, -3.2221609021867677)","(-7.52, -4.833569005680667)","(-7.43, -4.389265591381908)","(-9.81, -2.553538923921039)","(-9.85, -2.50180964821692)","(-9.85, -4.723724333857848)","(-9.37, -3.152849416813443)","(1.5, -4.5637046046916)","(-4.37, -4.207455620248347)","(-9.81, -7.543844276372717)","(-8.5, -5.722796876160175)","(1.12, -4.593419383903266)","(7.82, 0.11984412365868184)","(2.86, -2.6448864008638813)","(9.13, 0.671352980320803)","(-7.43, -6.930349806141761)","(2.14, -2.1344872921829943)","(-4.08, 1.162638834406824)","(-9.08, -7.469834372944979)","(7.82, -4.456726549573938)","(5.05, 0.5579963488069067)","(4.95, 1.5163582064302812)","(-9.17, -5.978853637371564)","(-8.4, -2.898395240205484)","(-8.4, -4.639022962885702)","(-8.4, -3.8721973647968206)",...,"(8.59, -1.7967645440127153)","(3.59, 0.36135340445714187)","(-6.84, -4.685828857354034)","(-9.03, -5.182769429743687)","(2.82, -2.6400259876507453)","(-1.36, -0.4586680158637653)","(-9.08, -6.6994589372540565)","(8.3, -0.2629803402791137)","(5.68, -0.809522590943238)","(-4.81, -4.749360919360776)","(99.0, -6.403368251532778)","(99.0, -1.4856214168498494)","(99.0, -4.850913579777779)","(99.0, -7.649601073174372)","(99.0, -5.070442144212578)","(99.0, -1.2720190401172022)","(99.0, -4.280256805310392)","(-9.42, -3.2968567348623212)","(99.0, -6.282063222131915)","(99.0, -4.428187604819685)","(99.0, -2.304635741119504)","(-7.72, -4.1119109485851375)","(99.0, -2.6310066483007546)","(99.0, -4.4130149296061525)","(99.0, -5.00229925054623)","(99.0, -3.168473250218546)","(99.0, -1.411036651115437)","(99.0, -1.5360785994829984)","(99.0, 1.6324365472577063)","(99.0, -4.784837223547947)","(2.82, -1.4362194074228365)","(99.0, -3.3299271630394025)","(99.0, -0.7051125963663316)","(99.0, -2.980742023130486)","(99.0, -3.4847235302616175)","(99.0, -3.0884718898022157)","(-5.63, -2.089720251528414)","(99.0, -6.116448576439491)","(99.0, -5.2296421917311315)","(99.0, -5.518415967428623)"
1,"(4.08, 3.6684775255986084)","(-0.29, 3.6572999313193195)","(6.36, 3.6396985905586794)","(4.37, 1.8878732837563161)","(-2.38, 1.9417718608979602)","(-9.66, 3.2676304367420355)","(-0.73, 2.0009398462340964)","(-5.34, 1.2004205901435492)","(8.88, 3.5497736020013684)","(9.22, 4.154486074387739)","(6.75, 4.264014680510981)","(8.64, 3.4454464316965145)","(4.42, 0.3414515329087469)","(7.43, 3.615267739836466)","(4.56, 0.2063880244977067)","(-0.97, -0.14023808300046195)","(4.66, 0.10765009273951355)","(-0.68, 1.4561622007713337)","(3.3, 1.6237824204527453)","(-1.21, 1.2447435582625292)","(0.87, 3.464583354157219)","(8.64, 4.367799065836857)","(8.35, 3.3867908897590286)","(9.17, 2.5581159585313586)","(0.05, 3.5308265063626685)","(7.57, 3.47525244953101)","(4.71, 3.455466754377939)","(0.87, 3.366977336994646)","(-0.39, 3.7147528421376466)","(6.99, 2.931315026249515)","(6.5, 3.475128514933462)","(-0.92, 3.5798321320472057)","(7.14, 2.6811287787586715)","(9.03, 3.3947052280279935)","(-1.8, 3.7195938480777833)","(0.73, 3.408680955866447)","(7.09, 2.496387057341331)","(3.4, 3.6774366327317636)","(-0.87, 3.2205499389555965)","(7.91, 3.42188422070089)",...,"(-6.7, 3.447531228410215)","(-3.35, 3.361765782475684)","(-9.03, 3.0849899146164907)","(4.47, 2.997554099207274)","(4.08, 3.443390960115326)","(-3.83, 3.7064399723380124)","(8.74, 3.363555723242084)","(1.12, 3.46704460742866